# PUMS Household Income in Phoenix  PUMAS

- https://www.census.gov/data/developers/data-sets/ ## FIX THIS

For households by income and race/ethnicity:
-  https://api.census.gov/data/2021/acs/acs1/pums/variables.html

In [1]:
import pandas as pd
import math
import numpy as np
import os

In [2]:
import get_pums as get
import utilcalcs as calc
from pums import *

In [3]:
#Search parameters
y1 = '2021'
#y0 = '2013'

sample = 'acs1'

phx_pumas = ['0400112','0400113','0400114','0400115','0400116','0400117',\
             '0400118','0400119','0400120','0400121','0400122','0400123',\
             '0400125','0400128','0400129']

data_cols = 'SERIALNO,ST,PUMA,HINCP,NP,WGTP'

## Get PUMA data

In [8]:
raw = get.get_puma(sample,y1,data_cols)

In [10]:
df = raw.copy()

In [11]:
df['GEO_ID'] = df['ST']+df['PUMA']
df = df[df.GEO_ID.isin(phx_pumas)]
df  = df.drop(['SERIALNO','ST','PUMA'],axis=1)
df = df[['GEO_ID']+[col for col in df.columns if col != 'GEO_ID']] #move id to first col
for col in df.columns[1:]: df[col] = df[col].astype(float)

In [13]:
df.NP.unique()

array([ 1.,  2.,  3.,  6.,  7.,  4.,  5.,  0.,  9.,  8., 10., 12., 11.,
       13.])

In [16]:
df.shape

(7489, 84)

In [ ]:
df['HHSz'] = pd.cut(x=df['HINCP'],bins=[1,2,3,4,5,6,7,np.inf],\
                   labels=['1','2','3','4','5','6','7p'])

In [17]:
print(len(df[df.HINCP==-60000]))

969


In [18]:
df.head()

,GEO_ID,HINCP,NP,WGTP,WGTP1,WGTP2,WGTP3,WGTP4,WGTP5,WGTP6,...,WGTP71,WGTP72,WGTP73,WGTP74,WGTP75,WGTP76,WGTP77,WGTP78,WGTP79,WGTP80
4,0400119,-60000.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33,0400129,-60000.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38,0400120,-60000.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40,0400116,-60000.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46,0400129,-60000.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### PUMA - median HHI by HH Size, total HHs by Size

In [ ]:
pum_size = get.get_puma('acs/acs1',y1,cols_inc_size)
pum_size = clean_table(pum_size)
pum_size = pum_size[pum_size['GEO_ID'].isin(phx_pumas)]

In [ ]:
pumas_rename = {'B19019_001E':'hh_tot_E','B19019_001M':'hh_tot_M','B19019_002E':'hh_1_E','B19019_002M':'hh_1_M',\
               'B19019_003E':'hh_2_E','B19019_003M':'hh_2_M','B19019_004E':'hh_3_E','B19019_004M':'hh_3_M',\
               'B19019_005E':'hh_4_E','B19019_005M':'hh_4_M','B19019_006E':'hh_5_E','B19019_006M':'hh_5_M',\
               'B19019_007E':'hh_6_E','B19019_007M':'hh_6_M','B19019_008E':'hh_7_E','B19019_008M':'hh_7_M'}

In [ ]:
pum_size.rename(columns=pumas_rename,inplace=True) 

In [ ]:
pum_size = geo_agg.sumgeo_cv(pum_size,'GEO_ID')

In [ ]:
pum_size

In [ ]:
pum_size.to_excel('output/medhhi_hhsize.xlsx',index=False)

with pd.ExcelWriter(f'output/HH_inc_2.xlsx') as writer:
    final.to_excel(writer, sheet_name="inc_bands", index=False)
    mean.to_excel(writer, sheet_name="mean_inc_uv", index=False)